# Consultas MDX Básicas con Atoti

En este notebook aprenderemos a realizar consultas MDX básicas utilizando Atoti y nuestro esquema universitario.

## 1. Configuración Inicial

Importamos las bibliotecas necesarias e iniciamos una sesión:

In [3]:
import atoti as tt
import pandas as pd
import numpy as np

session = tt.Session.start()

Welcome to Atoti 0.9.5!

By using this community edition, you agree with the license available at https://docs.atoti.io/latest/eula.html.
Browse the official documentation at https://docs.atoti.io.
Join the community at https://www.atoti.io/register.

Atoti collects telemetry data, which is used to help understand how to improve the product.
If you don't wish to send usage data, you can request a trial license at https://www.atoti.io/evaluation-license-request.

You can hide this message by setting the `ATOTI_HIDE_EULA_MESSAGE` environment variable to True.


## 2. Crear Datos de Ejemplo

Vamos a crear un conjunto de datos más complejo que incluya información sobre matrículas y rendimiento académico:

In [6]:
# Datos de matrículas y rendimiento
data = {
    'CursoAcademico': ['2023/2024'] * 12 + ['2022/2023'] * 12,
    'Centro': ['Facultad de Ciencias'] * 6 + ['Facultad de Letras'] * 6 + ['Facultad de Ciencias'] * 6 + ['Facultad de Letras'] * 6,
    'PlanEstudio': ['Grado en Informática', 'Grado en Matemáticas', 'Grado en Física', 
                   'Grado en Filología', 'Grado en Historia', 'Grado en Filosofía'] * 4,
    'NumeroMatriculados': [120, 80, 60, 90, 70, 50] * 4,
    'CreditosMatriculados': [7200, 4800, 3600, 5400, 4200, 3000] * 4,
    'TasaAprobados': [0.85, 0.82, 0.80, 0.90, 0.88, 0.85] * 4
}

df = pd.DataFrame(data)
df

,CursoAcademico,Centro,PlanEstudio,NumeroMatriculados,CreditosMatriculados,TasaAprobados
0,2023/2024,Facultad de Ciencias,Grado en Informática,120,7200,0.85
1,2023/2024,Facultad de Ciencias,Grado en Matemáticas,80,4800,0.82
2,2023/2024,Facultad de Ciencias,Grado en Física,60,3600,0.80
3,2023/2024,Facultad de Ciencias,Grado en Filología,90,5400,0.90
4,2023/2024,Facultad de Ciencias,Grado en Historia,70,4200,0.88
5,2023/2024,Facultad de Ciencias,Grado en Filosofía,50,3000,0.85
6,2023/2024,Facultad de Letras,Grado en Informática,120,7200,0.85
7,2023/2024,Facultad de Letras,Grado en Matemáticas,80,4800,0.82
8,2023/2024,Facultad de Letras,Grado en Física,60,3600,0.80
9,2023/2024,Facultad de Letras,Grado en Filología,90,5400,0.90


## 3. Crear Tabla y Cubo

Cargamos los datos en una tabla de Atoti y creamos un cubo:

In [8]:
# Crear tabla
academico_table = session.read_pandas(
    df,
    table_name="Academico",
    keys=["CursoAcademico", "Centro", "PlanEstudio"]  
)

# Crear cubo
cube = session.create_cube(academico_table)

## 4. Consultas MDX Básicas

### 4.1 Consulta Simple

Vamos a realizar una consulta simple para ver el número total de matriculados:

In [9]:
# Consulta simple
cube.query(cube.measures["NumeroMatriculados.SUM"])

,NumeroMatriculados.SUM
0,"1,880"


### 4.2 Consulta con una Dimensión

Ahora vamos a ver el número de matriculados por centro:

In [11]:
# Consulta por centro
print("\nNúmero de estudiantes por centro:")
print(
    cube.query(
        cube.measures["NumeroMatriculados.SUM"],
        levels=[cube.levels[("Academico", "Centro", "Centro")]]
    )
)


Número de estudiantes por centro:
                     NumeroMatriculados.SUM
Centro                                     
Facultad de Ciencias                    940
Facultad de Letras                      940


### 4.3 Consulta con Múltiples Dimensiones

Vamos a ver el número de matriculados por centro y plan de estudios:

In [12]:
# Consulta por centro y plan de estudios
print("\nNúmero de estudiantes por centro y plan de estudios:")
print(
    cube.query(
        cube.measures["NumeroMatriculados.SUM"],
        levels=[
            cube.levels[("Academico", "Centro", "Centro")],
            cube.levels[("Academico", "PlanEstudio", "PlanEstudio")]
        ]
    )
)



Número de estudiantes por centro y plan de estudios:
                                          NumeroMatriculados.SUM
Centro               PlanEstudio                                
Facultad de Ciencias Grado en Filología                      180
                     Grado en Filosofía                      100
                     Grado en Física                         120
                     Grado en Historia                       140
                     Grado en Informática                    240
                     Grado en Matemáticas                    160
Facultad de Letras   Grado en Filología                      180
                     Grado en Filosofía                      100
                     Grado en Física                         120
                     Grado en Historia                       140
                     Grado en Informática                    240
                     Grado en Matemáticas                    160


### 4.4 Consulta con Filtros

Vamos a filtrar los datos para ver solo los resultados de un curso académico específico:

In [13]:
# Consulta con filtro
print("\nNúmero de estudiantes por curso académico:")
print(
    cube.query(
        cube.measures["NumeroMatriculados.SUM"],
        levels=[cube.levels[("Academico", "CursoAcademico", "CursoAcademico")]]
    )
)


Número de estudiantes por curso académico:
               NumeroMatriculados.SUM
CursoAcademico                       
2022/2023                         940
2023/2024                         940


### 4.5 Consulta con Múltiples Medidas

Vamos a ver tanto el número de matriculados como la nota media:

In [17]:
print("\nNúmero de estudiantes y créditos por centro y plan de estudios:")
print(
    cube.query(
        cube.measures["NumeroMatriculados.SUM"],
        cube.measures["CreditosMatriculados.SUM"],
        levels=[
            cube.levels[("Academico", "Centro", "Centro")],
            cube.levels[("Academico", "PlanEstudio", "PlanEstudio")]
        ]
    )
)


Número de estudiantes y créditos por centro y plan de estudios:
                                          NumeroMatriculados.SUM  \
Centro               PlanEstudio                                   
Facultad de Ciencias Grado en Filología                      180   
                     Grado en Filosofía                      100   
                     Grado en Física                         120   
                     Grado en Historia                       140   
                     Grado en Informática                    240   
                     Grado en Matemáticas                    160   
Facultad de Letras   Grado en Filología                      180   
                     Grado en Filosofía                      100   
                     Grado en Física                         120   
                     Grado en Historia                       140   
                     Grado en Informática                    240   
                     Grado en Matemáticas          

## 5. Crear Medidas Calculadas

Vamos a crear algunas medidas calculadas útiles:

In [21]:
print(cube.measures)

{'CreditosMatriculados.MEAN': <atoti.measure.Measure object at 0x0000028ABB9BC4A0>, 'contributors.COUNT': <atoti.measure.Measure object at 0x0000028ABB9BD850>, 'NumeroMatriculados.SUM': <atoti.measure.Measure object at 0x0000028ABB9BD970>, 'TasaAprobados.SUM': <atoti.measure.Measure object at 0x0000028ABB9BEBA0>, 'CreditosMatriculados.SUM': <atoti.measure.Measure object at 0x0000028ABA8A2CF0>, 'update.TIMESTAMP': <atoti.measure.Measure object at 0x0000028ABA8A6420>, 'TasaAprobados.MEAN': <atoti.measure.Measure object at 0x0000028ABA8A63C0>, 'NumeroMatriculados.MEAN': <atoti.measure.Measure object at 0x0000028ABA8A5C40>}


In [22]:
# Medida para el total de créditos
cube.measures["Total Creditos"] = tt.agg.sum(
    cube.measures["CreditosMatriculados.SUM"],
    scope=tt.OriginScope(levels=[cube.levels[("Academico", "CursoAcademico", "CursoAcademico")]])
)

# Medida para la tasa de aprobados ponderada
cube.measures["Tasa Aprobados Ponderada"] = tt.agg.mean(
    cube.measures["TasaAprobados.MEAN"] * cube.measures["NumeroMatriculados.SUM"],
    scope=tt.OriginScope(levels=[cube.levels[("Academico", "CursoAcademico", "CursoAcademico")]])
)

# Medida para la media de créditos por estudiante
cube.measures["Creditos por Estudiante"] = tt.agg.mean(
    cube.measures["CreditosMatriculados.SUM"] / cube.measures["NumeroMatriculados.SUM"],
    scope=tt.OriginScope(levels=[cube.levels[("Academico", "CursoAcademico", "CursoAcademico")]])
)

# Medida para el total de aprobados
cube.measures["Total Aprobados"] = tt.agg.sum(
    cube.measures["TasaAprobados.SUM"] * cube.measures["NumeroMatriculados.SUM"],
    scope=tt.OriginScope(levels=[cube.levels[("Academico", "CursoAcademico", "CursoAcademico")]])
)

# Medida para la eficiencia docente
cube.measures["Eficiencia Docente"] = tt.agg.mean(
    cube.measures["TasaAprobados.MEAN"] * cube.measures["CreditosMatriculados.MEAN"],
    scope=tt.OriginScope(levels=[cube.levels[("Academico", "CursoAcademico", "CursoAcademico")]])
)

## 6. Consultas con Medidas Calculadas

Vamos a usar nuestras nuevas medidas calculadas:

In [23]:
# Consulta con medidas calculadas por centro
print("\nMedidas calculadas por centro:")
print(
    cube.query(
        cube.measures["Total Creditos"],
        cube.measures["Tasa Aprobados Ponderada"],
        cube.measures["Creditos por Estudiante"],
        levels=[cube.levels[("Academico", "Centro", "Centro")]]
    )
)

# Consulta con medidas calculadas por centro y plan de estudios
print("\nMedidas calculadas por centro y plan de estudios:")
print(
    cube.query(
        cube.measures["Total Creditos"],
        cube.measures["Tasa Aprobados Ponderada"],
        cube.measures["Creditos por Estudiante"],
        cube.measures["Total Aprobados"],
        cube.measures["Eficiencia Docente"],
        levels=[
            cube.levels[("Academico", "Centro", "Centro")],
            cube.levels[("Academico", "PlanEstudio", "PlanEstudio")]
        ]
    )
)

# Consulta con medidas calculadas por curso académico
print("\nMedidas calculadas por curso académico:")
print(
    cube.query(
        cube.measures["Total Creditos"],
        cube.measures["Tasa Aprobados Ponderada"],
        cube.measures["Creditos por Estudiante"],
        cube.measures["Total Aprobados"],
        cube.measures["Eficiencia Docente"],
        levels=[cube.levels[("Academico", "CursoAcademico", "CursoAcademico")]]
    )
)


Medidas calculadas por centro:
                     Total Creditos Tasa Aprobados Ponderada  \
Centro                                                         
Facultad de Ciencias         56,400                   399.50   
Facultad de Letras           56,400                   399.50   

                     Creditos por Estudiante  
Centro                                        
Facultad de Ciencias                   60.00  
Facultad de Letras                     60.00  

Medidas calculadas por centro y plan de estudios:
                                          Total Creditos  \
Centro               PlanEstudio                           
Facultad de Ciencias Grado en Filología           10,800   
                     Grado en Filosofía            6,000   
                     Grado en Física               7,200   
                     Grado en Historia             8,400   
                     Grado en Informática         14,400   
                     Grado en Matemáticas          9

## Ejercicios Prácticos

1. Crea una consulta que muestre el número de matriculados y la nota media por asignatura
2. Crea una medida calculada que muestre la relación entre créditos y número de matriculados
3. Realiza una consulta que compare el rendimiento entre diferentes cursos académicos
4. Crea una consulta que muestre las tasas de aprobados por centro y plan de estudios